## Alpha/Beta Diversity for EAC Progresson
Date: 5/15/23  
Goal: Get basic micorbial alpha/beta diversity  
Will likely expand upon this more before the paper, but for now just getting an idea of what our samples look like

### Imports

In [3]:
import pandas as pd
import qiime2 as q2
from qiime2 import Artifact, Metadata
from biom import load_table

from qiime2.plugins.taxa.visualizers import barplot
from qiime2.plugins.diversity.pipelines import alpha
from qiime2.plugins.diversity.visualizers import alpha_group_significance
from qiime2.plugins.feature_table.methods import merge
from qiime2.plugins.emperor.visualizers import biplot

In [4]:
#WOL Taxonomy in FeatureData[Taxonomy] Form
wol_taxonomy = Artifact.import_data('FeatureData[Taxonomy]', '/Users/cguccion/Dropbox/Storage/HelpfulLabDocs/taxonomy_trees/WOL/lineages.txt', 'HeaderlessTSVTaxonomyFormat')
wol_taxonomy

<artifact: FeatureData[Taxonomy] uuid: ee8ead32-f0a3-4bc0-832e-2066881b28e6>

### Functions

In [5]:
def combine_qza_meta(fn_list, combo_meta_col=['sample_name', 'study_label']):
    
    '''Combine all imported fn's'''
    ft = Artifact.load('processed_data/qza/' + fn_list[0] + '.qza')
    
    meta = pd.read_csv('processed_data/metadata/metadata_' + fn_list[0] + '.tsv', sep='\t', index_col=0)
    meta['study_label'] = fn_list[0]
    
    count=0
    for fn in fn_list[1:]:
        
        #Merge taxonomy tables
        current_ft = Artifact.load('processed_data/qza/' + fn + '.qza')
        ft, = merge([ft, current_ft])
        
        #Merge metadata
        current_meta = pd.read_csv('processed_data/metadata/metadata_' + fn + '.tsv', sep='\t', index_col=0)
        current_meta['study_label'] = fn
        meta = pd.merge(meta, current_meta, how = 'outer', on= combo_meta_col, suffixes=('_' + str(count), '_' + str(count+1)))
    
        count+=1
        
    #meta = pd.concat([meta['study_label'], meta.drop('study_label', axis=1)], axis=1)
    meta = meta[['study_label']]
    
    meta = q2.Metadata(meta)
    
    #print(meta)
    
    return(ft, meta)

In [63]:
def multi_taxonomy_barplot(fn_list, name, taxonomy=wol_taxonomy, combo_meta_col =['sample_name', 'study_label']):
    
    ft, meta = combine_qza_meta(fn_list, combo_meta_col=combo_meta_col)
    
    '''Create taxonomy barplots'''
    barplot_output = barplot(table=ft, taxonomy=taxonomy, metadata=meta)
    barplot_output_v = barplot_output.visualization
    barplot_output_v.save('outputs/taxonomy_barplots/barplot_' + name + '.qzv')
        

In [3]:
def taxonomy_barplot(fn, taxonomy=wol_taxonomy):
    '''Create taxonomy barplots'''
    
    ft = Artifact.load('processed_data/qza/' + fn + '.qza')
    meta = q2.Metadata(pd.read_csv('processed_data/metadata/metadata_' + fn + '.tsv', sep='\t', index_col=0))
    
    barplot_output = barplot(table=ft, taxonomy=taxonomy, metadata=meta)
    barplot_output_v = barplot_output.visualization
    barplot_output_v.save('outputs/taxonomy_barplots/barplot_' + fn + '.qzv')

In [89]:
def multi_basic_alpha(fn_list, name, metric = 'observed_features', combo_meta_col =['sample_name', 'study_label']):
    
    ft, meta = combine_qza_meta(fn_list, combo_meta_col=combo_meta_col)
    
    alpha_vector, = alpha(table= ft, metric=metric)
    alpha_output = alpha_group_significance(alpha_diversity = alpha_vector, metadata = meta)
    alpha_output_v = alpha_output.visualization
    alpha_output_v.save('outputs/alpha_plots/alpha_' + name + '_' + metric + '.qzv')

In [88]:
def basic_alpha(fn, metric = 'observed_features'):
    '''Run basic alpha diversity'''
    
    ft = Artifact.load('processed_data/qza/' + fn + '.qza')
    meta = q2.Metadata(pd.read_csv('processed_data/metadata/metadata_' + fn + '.tsv', sep='\t', index_col=0))
    
    alpha_vector, = alpha(table= ft, metric=metric)
    alpha_output = alpha_group_significance(alpha_diversity = alpha_vector, metadata = meta)
    alpha_output_v = alpha_output.visualization
    alpha_output_v.save('outputs/alpha_plots/alpha_' + fn + '_' + metric + '.qzv')

In [7]:
def beta_decoide(fn_list, name, metric='study_label', permutations=999):
    
    '''Calculates beta diversity for unweighted 
    & weighted unifraq

    Parameters
    ---------
    ft: q2 FeatureTable[Frequency] object
        Qiime2 taxonomy object
    
    meta: pd df
        metadata
        
    metric: str
        The column in sample_meta used to
        calculate beta diversity across 
        samples
        
    return_biplot: bool (False)
        Returns the biplot instead 
        of the permaonva ouputs 
    
    Returns
    -------
    beta_result_d: q2 Visulization object
        Qiime2 visulaization object of 
        decoide
    
    Notes
    -----
    '''
    
    ft, meta = combine_qza_meta(fn_list)
    
    '''
    #Save ft for command line processing
    ft.save('deicode_processing.qza')
    
    #Run deicode from command line using qza created above
    !qiime deicode rpca --i-table deicode_processing.qza --o-biplot deicode_biplot.qza --o-distance-matrix deicode_distance_test.qza
    
    #--p-min-sample-count 500 --p-min-feature-count $decoide_min_feature_count
    '''
    
    #Import biplot back into python
    rpca_biplot = Artifact.load('deicode_biplot.qza')
    
    #Import biplot back into python
    rpca_distance_matrix = Artifact.load('deicode_distance_test.qza')
    
    #Create emperor visualization from the biplot result
    rpca_biplot_emperor = biplot(biplot = rpca_biplot, sample_metadata = meta)
    
    rpca_biplot_emperor_v = rpca_biplot_emperor.visualization
    rpca_biplot_emperor_v.save('outputs/RPCA/RPCA_' + name + '_' + '.qzv')
    
    #Calculate permanova 
    beta_result_d = diversity.actions.beta_group_significance(distance_matrix=rpca_distance_matrix,
                                                              metadata=meta.get_column(metric),
                                                              method = 'permanova', pairwise = True,
                                                              permutations=permutations)
    
    
    beta_result_v = beta_result_d.visualization
    beta_result_v.save('outputs/RPCA/permanova_' + name + '_' + '.qzv')
    
    return(beta_result_d)  

### Taxonomy Barplots

In [6]:
taxonomy_barplot('normal_genome_WOL')

In [7]:
taxonomy_barplot('GERD_genome_WOL')

In [8]:
taxonomy_barplot('BE_genome_WOL')

In [9]:
taxonomy_barplot('EAC_genome_WOL')

In [4]:
taxonomy_barplot('BE_Ross_paired_genome_WOL')

In [5]:
taxonomy_barplot('EAC_Ross_paired_genome_WOL')

In [9]:
taxonomy_barplot('BE_Prog_T2_genome_WOL')

#### Run Jan 17, 2024

In [5]:
taxonomy_barplot('normal_genome_WOL_scrubbed')

In [6]:
taxonomy_barplot('GERD_genome_WOL_scrubbed')

In [7]:
taxonomy_barplot('BE_NP_genome_WOL_scrubbed')

In [8]:
taxonomy_barplot('BE_Prog_T1_genome_WOL_scrubbed')

In [9]:
taxonomy_barplot('BE_Prog_T2_genome_WOL_scrubbed')

In [11]:
taxonomy_barplot('BE_exact_Ross_paired_genome_WOL_scrubbed')

In [12]:
taxonomy_barplot('EAC_ICGC_genome_WOL_scrubbed')

In [76]:
multi_taxonomy_barplot(['normal_genome_WOL_scrubbed', 'GERD_genome_WOL_scrubbed',
                        'BE_NP_genome_WOL_scrubbed', 'BE_Prog_T1_genome_WOL_scrubbed',
                       'BE_Prog_T2_genome_WOL_scrubbed', 'BE_exact_Ross_paired_genome_WOL_scrubbed',
                       'EAC_ICGC_genome_WOL_scrubbed', 'EAC_Ross_paired_genome_WOL_scrubbed'],
                      'Combo-Progression_WOL_scrubbed')

### Alpha Plots

In [24]:
basic_alpha('BE_Ross_paired_genome_WOL')

In [25]:
basic_alpha('EAC_Ross_paired_genome_WOL')

In [23]:
basic_alpha('BE_Prog_T2_genome_WOL')

In [26]:
basic_alpha('EAC_genome_WOL')

#### Run Jan 17, 2024

In [90]:
multi_basic_alpha(['normal_genome_WOL_scrubbed', 'GERD_genome_WOL_scrubbed',
                        'BE_NP_genome_WOL_scrubbed', 'BE_Prog_T1_genome_WOL_scrubbed',
                       'BE_Prog_T2_genome_WOL_scrubbed', 'BE_exact_Ross_paired_genome_WOL_scrubbed',
                       'EAC_ICGC_genome_WOL_scrubbed', 'EAC_Ross_paired_genome_WOL_scrubbed'],
                      'Combo-Progression_WOL_scrubbed')

In [91]:
multi_basic_alpha(['normal_genome_WOL_scrubbed', 'GERD_genome_WOL_scrubbed',
                        'BE_NP_genome_WOL_scrubbed', 'BE_Prog_T1_genome_WOL_scrubbed',
                       'BE_Prog_T2_genome_WOL_scrubbed', 'BE_exact_Ross_paired_genome_WOL_scrubbed',
                       'EAC_ICGC_genome_WOL_scrubbed', 'EAC_Ross_paired_genome_WOL_scrubbed'],
                      'Combo-Progression_WOL_scrubbed', 'shannon')

#### RPCA

In [12]:
beta_decoide(['normal_genome_WOL_scrubbed', 'GERD_genome_WOL_scrubbed',
                        'BE_NP_genome_WOL_scrubbed', 'BE_Prog_T1_genome_WOL_scrubbed',
                       'BE_Prog_T2_genome_WOL_scrubbed', 'BE_exact_Ross_paired_genome_WOL_scrubbed',
                       'EAC_ICGC_genome_WOL_scrubbed', 'EAC_Ross_paired_genome_WOL_scrubbed'],
                      'Combo-Progression_WOL_scrubbed')

KeyError: (3.0, 0.0, 'BE_Prog_T2_genome_WOL_scrubbed (n=1817)', '.15', 'top', 'center', -3728358448861741636, 90, None, False, 72, <weakref at 0x7fedff7de9a0; to 'MixedModeRenderer' at 0x7fedff7e5400>, 1.2)